The idea of that notebook is to have a reproducible benchmark working with the masters version of `pysap` and `modopt` in April 2019. The benchmark will be on speed, for both sparkling and uniform random acquisition schemes, with a brain phantom, for decimated wavelets.

The benchmark will future the algorithms of the abstract submission for SPARS 2019.

Special attention must be taken for Condat. We might need to qualify the benchmark with non reproducible experiments where we set the norm of the linear operator. We also need to take care of the relaxation factor.

In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

# Third party import
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import imread
from tqdm import tqdm_notebook

# Package import
from modopt.math.metrics import ssim
import pysap
from pysap.data import get_sample_data
from mri.numerics.reconstruct import sparse_rec_fista, sparse_rec_pogm, sparse_rec_condatvu
from mri.reconstruct.fourier import NFFT
from mri.reconstruct.utils import generate_operators, convert_mask_to_locations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#  Loading input data

In [8]:
image = get_sample_data("mri-slice-nifti")
original_image_data = np.copy(image.data)
image.data += np.random.randn(*image.shape) * 20.
image.data = image.data.astype(np.complex128)
mask = get_sample_data("mri-mask")

In [9]:
non_cartesian = True
opt_results = {}

# Generate the kspace 
From the 2D brain slice and the acquistion mask, we generate the acquisition measurments, the observed kspace. We then reconstruct the zero order solution.

In [10]:
# Generate the subsampled kspace
if non_cartesian:
    kspace_loc = convert_mask_to_locations(mask.data)
    fourier_op = NFFT(samples=kspace_loc, shape=image.shape)
    kspace_data = fourier_op.op(image.data)

    # Zero order solution
    image_rec0 = pysap.Image(data=fourier_op.adj_op(kspace_data), metadata=image.metadata)
else:
    kspace_mask = np.fft.ifftshift(mask.data)
    kspace_data = np.fft.fft2(image.data, norm="ortho") * kspace_mask

    # Get the locations of the kspace samples
    kspace_loc = convert_mask_to_locations(kspace_mask)

    # Zero order solution
    image_rec0 = pysap.Image(data=np.fft.ifft2(kspace_data, norm="ortho"), metadata=image.metadata)

# FISTA optimization
We now want to refine the zero order solution using a FISTA optimization.

In [11]:
# Generate operators
gradient_op, linear_op, prox_op, cost_op = generate_operators(
    data=kspace_data,
    wavelet_name="Db4",
    samples=kspace_loc,
    nb_scales=4,
    non_cartesian=non_cartesian,
    uniform_data_shape=image.shape,
    gradient_space="synthesis",
)

ModuleNotFoundError: No module named 'astro'

In [ ]:
def objective_cost(x):
    return data_fidelity(x) + sparsity(x)

def sparsity(x):
    return mu * np.sum(np.abs(x))

def data_fidelity(x):
    return 0.5 * np.linalg.norm(gradient_op.op(x) - kspace_data)**2

In [ ]:
def nrmse(x):
    return np.linalg.norm(x - original_image_data) / np.mean(original_image_data)

In [ ]:
metrics_ = {
    "cost": {"metric": objective_cost, "mapping": {"z_new": "x"}, "cst_kwargs": {}, "early_stopping": False},
    "nrmse": {"metric": nrmse, "mapping": {"x_new": "x"}, "cst_kwargs": {}, "early_stopping": False},
}

In [ ]:
mu = 80

In [ ]:
max_iter = 1000
convergence_pot = max_iter >= 500

In [ ]:
# Start the FISTA reconstruction
cost_op = None
x_final, transform, costs, metrics = sparse_rec_fista(
    gradient_op,
    linear_op,
    prox_op,
    cost_op,
    mu=mu,
    lambda_init=1.0,
    max_nb_of_iter=max_iter,
    verbose=1,
    metrics=metrics_,
    metric_call_period=20,
)
image_rec = pysap.Image(data=np.abs(x_final))

In [ ]:
if convergence_pot:
    opt_results[mu] = {
        "nrmse": metrics['nrmse']['values'][-1],
        "cost": metrics['cost']['values'][-1],
    }
    print(opt_results)

In [ ]:
max_iter = 3

In [ ]:
# FISTA rec
params = {
    r"FISTA-BT": ({}, "black"),
    r"FISTA-CD, $a = 20$": ({"a_cd": 20}, "blue"),    
    r"Rada-FISTA": (
        {"p_lazy": (1/30), "q_lazy": (1/10), "restart_strategy": "adaptive", "xi_restart": 0.96}, 
        "orange",
    ),
    r"greedy FISTA": (
        {"restart_strategy": "greedy", "xi_restart": 0.96, "s_greedy": 1.1}, 
        "violet",
    ),
}

res_param = {}
for param_name, (param, _) in tqdm_notebook(params.items()):
    _, _, _, metrics = sparse_rec_fista(
        gradient_op,
        linear_op,
        prox_op,
        None,
        mu=mu,
        lambda_init=1.0,
        max_nb_of_iter=max_iter,
        verbose=0,
        metrics=metrics_,
        metric_call_period=1,
        **param,
    )
    res_param[param_name] = {
        "cost": np.array(metrics['cost']['values']),
        "nrmse": np.array(metrics['nrmse']['values']),
    }

# POGM rec
pogm_name = r"POGM'"
params[pogm_name] = (None, "red")

_, _, _, metrics = sparse_rec_pogm(
    gradient_op,
    linear_op,
    prox_op,
    mu=mu,
    max_nb_of_iter=max_iter,
    verbose=0,
    metrics=metrics_,
    metric_call_period=1,
)
res_param[pogm_name] = {
    "cost": np.array(metrics['cost']['values']),
    "nrmse": np.array(metrics['nrmse']['values']),
}

In [ ]:
sparse_rec_pogm

In [ ]:
metric_name = "cost"
plt.figure(figsize=(9, 5))
for param_name, param_res in res_param.items():
    plt.plot(
        np.log10(np.abs(np.array(param_res[metric_name] - opt_results[mu][metric_name]))), 
        label=param_name, 
        color=params[param_name][1],
    )
plt.legend()
plt.title(r"Cost evolution for different restarting strategies")
plt.xlabel(r"$k$")
plt.ylabel(r"$\log_{10}($cost$(k) -$ cost$*)$")